In [1]:
!pip install tensorflow-gpu==1.15.0
!pip install h5py=='2.9.0'

     |████████████████████████████████| 411.5MB 39kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 3.8MB 31.8MB/s 
     |████████████████████████████████| 512kB 35.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=be082dbe6dc39b2d1819588b6f3c7426b6de02d7060594d47432c00b392f8cdc
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorflow-estimator<2.6.0,>=2.5.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

In [2]:
!git clone https://github.com/JiangnanH/PrivacyInternship
%cd /content/PrivacyInternship/wgan-gp

Cloning into 'PrivacyInternship'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 263 (delta 116), reused 217 (delta 70), pack-reused 0
Receiving objects: 100% (263/263), 47.33 MiB | 35.15 MiB/s, done.
Resolving deltas: 100% (116/116), done.
/content/PrivacyInternship/wgan-gp


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/PrivacyInternship/wgan-gp

/content/PrivacyInternship/wgan-gp


In [5]:
import os

import numpy as np # linear algebra
np.random.seed(2142)
from tensorflow import set_random_seed
set_random_seed(2)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Convolution2D, Input,Activation, ZeroPadding2D, MaxPooling2D, Flatten, Add
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy as scc
from tensorflow.keras.losses import binary_crossentropy as bc
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#from sklearn import svm

from tflib.nist import load_nist_images

In [6]:
def load_hsf_nist(hsf_num, num_images=1000, resize_width=28, resize_height=28):
    with open(os.path.join('/content/drive/MyDrive/Internships/data/nist', 'HSF_{}_images.npy'.format(hsf_num)),'rb') as f:
        images = load_nist_images(np.load(f), num_images, resize_width=resize_width, resize_height=resize_height)
    with open(os.path.join('/content/drive/MyDrive/Internships/data/nist', 'HSF_{}_labels.npy'.format(hsf_num)),'rb') as f:
        labels = np.load(f)[:num_images]
    images = images.reshape(num_images, resize_width, resize_height, 1)
    return (images,labels)

def load_nist_by_origin(num_images=30000, source='all', resize_width=28, resize_height=28):    
    if source == 'all':
        num = int(num_images/3)
        images1,labels1 = load_hsf_nist(4, num, resize_width, resize_height)
        images2,labels2 = load_hsf_nist(6, num, resize_width, resize_height)
        images3,labels3 = load_hsf_nist(1, num, resize_width, resize_height)
        output=(np.concatenate((images1,images2,images3)),np.concatenate((labels1,labels2,labels3)))
    elif source == 'census':
        num = int(num_images/2)
        images2,labels2 = load_hsf_nist(6, num, resize_width, resize_height)
        images3,labels3 = load_hsf_nist(1, num, resize_width, resize_height)
        output=(np.concatenate((images2,images3)),np.concatenate((labels2,labels3)))
    elif source == 'high-school':
        num = int(num_images)
        output = load_hsf_nist(4, num, resize_width, resize_height)
    return output

def load_high_school_vs_non(num_images=30000, resize_width=28, resize_height=28):
    num = int(num_images/2)

    images1,_ = load_hsf_nist(4, num, resize_width, resize_height)
    images1 = images1.reshape(num,resize_width, resize_height,1)
    labels1 = np.array([1 for l in range(images1.shape[0])]) 
       
    images2,_ = load_hsf_nist(1, num, resize_width, resize_height)
    images2 = images2.reshape(num,resize_width, resize_height,1)
    labels2 = np.array([0 for l in range(images2.shape[0])])
    output=(np.concatenate((images1,images2)),np.concatenate((labels1,labels2)))
    return output

def get_argmax_labels(preds):
    predicted_labels=[]
    for array in preds:
        argmax=np.where(array == np.amax(array))[0][0]
        predicted_labels.append(argmax)
    return predicted_labels

def grey2RGB(gray):
    return cv2.cvtColor(gray.astype('float32'), cv2.COLOR_GRAY2BGR)

In [7]:
def get_resnet():
    # In order to make things less confusing, all layers have been declared first, and then used
    
    # declaration of layers
    input_img = Input((28, 28, 1), name='input_layer')
    zeroPad1 = ZeroPadding2D((1,1), name='zeroPad1')
    zeroPad1_2 = ZeroPadding2D((1,1), name='zeroPad1_2')
    layer1 = Convolution2D(6, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv')
    layer1_2 = Convolution2D(16, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv2')
    zeroPad2 = ZeroPadding2D((1,1), name='zeroPad2')
    zeroPad2_2 = ZeroPadding2D((1,1), name='zeroPad2_2')
    layer2 = Convolution2D(6, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv')
    layer2_2 = Convolution2D(16, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv2')


    zeroPad3 = ZeroPadding2D((1,1), name='zeroPad3')
    zeroPad3_2 = ZeroPadding2D((1,1), name='zeroPad3_2')
    layer3 = Convolution2D(6, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv')
    layer3_2 = Convolution2D(16, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv2')

    layer4 = Dense(64, activation='relu', kernel_initializer='he_uniform', name='dense1')
    layer5 = Dense(32, activation='relu', kernel_initializer='he_uniform', name='dense2')

    final = Dense(10, activation='softmax', kernel_initializer='he_uniform', name='classifier')
    
    # declaration completed
    
    first = zeroPad1(input_img)
    second = layer1(first)
    second = BatchNormalization(axis=1, name='major_bn')(second)
    second = Activation('relu', name='major_act')(second)

    third = zeroPad2(second)
    third = layer2(third)
    third = BatchNormalization(axis=1, name='l1_bn')(third)
    third = Activation('relu', name='l1_act')(third)

    third = zeroPad3(third)
    third = layer3(third)
    third = BatchNormalization(axis=1, name='l1_bn2')(third)
    third = Activation('relu', name='l1_act2')(third)


    #res = merge([third, second], mode='sum', name='res')
    res = Add(name='res')([third, second])


    first2 = zeroPad1_2(res)
    second2 = layer1_2(first2)
    second2 = BatchNormalization(axis=1, name='major_bn2')(second2)
    second2 = Activation('relu', name='major_act2')(second2)


    third2 = zeroPad2_2(second2)
    third2 = layer2_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn')(third2)
    third2 = Activation('relu', name='l2_act')(third2)

    third2 = zeroPad3_2(third2)
    third2 = layer3_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn2')(third2)
    third2 = Activation('relu', name='l2_act2')(third2)

    #res2 = merge([third2, second2], mode='sum', name='res2')
    res2 = Add(name='res2')([third2, second2])
    res2 = Flatten(name='flatten')(res2)

    res2 = layer4(res2)
    res2 = Dropout(0.4, name='dropout1')(res2)
    res2 = layer5(res2)
    res2 = Dropout(0.4, name='dropout2')(res2)
    res2 = final(res2)
    model = Model(inputs=input_img, outputs=res2)

    if binary_classification:
        metrics = ['accuracy', AUC()]
        loss = bc
        final = Dense(1, activation='sigmoid', kernel_initializer='he_uniform', name='classifier')
    else:
        metrics = ['accuracy']
        loss = scc
        final = Dense(10, activation='softmax', kernel_initializer='he_uniform', name='classifier')
    
    
    sgd = SGD(decay=0., lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=scc, optimizer=sgd, metrics=['accuracy'])
    return model

def get_resnet_binary():
    # In order to make things less confusing, all layers have been declared first, and then used
    
    # declaration of layers
    input_img = Input((28, 28, 1), name='input_layer')
    zeroPad1 = ZeroPadding2D((1,1), name='zeroPad1')
    zeroPad1_2 = ZeroPadding2D((1,1), name='zeroPad1_2')
    layer1 = Convolution2D(6, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv')
    layer1_2 = Convolution2D(16, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv2')
    zeroPad2 = ZeroPadding2D((1,1), name='zeroPad2')
    zeroPad2_2 = ZeroPadding2D((1,1), name='zeroPad2_2')
    layer2 = Convolution2D(6, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv')
    layer2_2 = Convolution2D(16, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv2')


    zeroPad3 = ZeroPadding2D((1,1), name='zeroPad3')
    zeroPad3_2 = ZeroPadding2D((1,1), name='zeroPad3_2')
    layer3 = Convolution2D(6, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv')
    layer3_2 = Convolution2D(16, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv2')

    layer4 = Dense(64, activation='relu', kernel_initializer='he_uniform', name='dense1')
    layer5 = Dense(32, activation='relu', kernel_initializer='he_uniform', name='dense2')

    final = Dense(1, activation='sigmoid', kernel_initializer='he_uniform', name='classifier')
    
    # declaration completed
    
    first = zeroPad1(input_img)
    second = layer1(first)
    second = BatchNormalization(axis=1, name='major_bn')(second)
    second = Activation('relu', name='major_act')(second)

    third = zeroPad2(second)
    third = layer2(third)
    third = BatchNormalization(axis=1, name='l1_bn')(third)
    third = Activation('relu', name='l1_act')(third)

    third = zeroPad3(third)
    third = layer3(third)
    third = BatchNormalization(axis=1, name='l1_bn2')(third)
    third = Activation('relu', name='l1_act2')(third)


    #res = merge([third, second], mode='sum', name='res')
    res = Add(name='res')([third, second])


    first2 = zeroPad1_2(res)
    second2 = layer1_2(first2)
    second2 = BatchNormalization(axis=1, name='major_bn2')(second2)
    second2 = Activation('relu', name='major_act2')(second2)


    third2 = zeroPad2_2(second2)
    third2 = layer2_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn')(third2)
    third2 = Activation('relu', name='l2_act')(third2)

    third2 = zeroPad3_2(third2)
    third2 = layer3_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn2')(third2)
    third2 = Activation('relu', name='l2_act2')(third2)

    #res2 = merge([third2, second2], mode='sum', name='res2')
    res2 = Add(name='res2')([third2, second2])
    res2 = Flatten()(res2)

    res2 = layer4(res2)
    res2 = Dropout(0.4, name='dropout1')(res2)
    res2 = layer5(res2)
    res2 = Dropout(0.4, name='dropout2')(res2)
    res2 = final(res2)
    model = Model(inputs=input_img, outputs=res2)
    
    
    sgd = SGD(decay=0., lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=bc, optimizer=sgd, metrics=['accuracy',AUC()])
    return model

def get_resnet(binary_classification=False):     
    # In order to make things less confusing, all layers have been declared first, and then used
    
    # declaration of layers
    input_img = Input((28, 28, 1), name='input_layer')
    zeroPad1 = ZeroPadding2D((1,1), name='zeroPad1')
    zeroPad1_2 = ZeroPadding2D((1,1), name='zeroPad1_2')
    layer1 = Convolution2D(6, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv')
    layer1_2 = Convolution2D(16, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', name='major_conv2')
    zeroPad2 = ZeroPadding2D((1,1), name='zeroPad2')
    zeroPad2_2 = ZeroPadding2D((1,1), name='zeroPad2_2')
    layer2 = Convolution2D(6, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv')
    layer2_2 = Convolution2D(16, (3, 3), strides=(1,1), kernel_initializer='he_uniform', name='l1_conv2')


    zeroPad3 = ZeroPadding2D((1,1), name='zeroPad3')
    zeroPad3_2 = ZeroPadding2D((1,1), name='zeroPad3_2')
    layer3 = Convolution2D(6, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv')
    layer3_2 = Convolution2D(16, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', name='l2_conv2')

    layer4 = Dense(64, activation='relu', kernel_initializer='he_uniform', name='dense1')
    layer5 = Dense(32, activation='relu', kernel_initializer='he_uniform', name='dense2')

    if binary_classification:
        final = Dense(1, activation='sigmoid', kernel_initializer='he_uniform', name='classifier')
    else:
        final = Dense(10, activation='softmax', kernel_initializer='he_uniform', name='classifier')
    
    # declaration completed
    
    first = zeroPad1(input_img)
    second = layer1(first)
    second = BatchNormalization(axis=1, name='major_bn')(second)
    second = Activation('relu', name='major_act')(second)

    third = zeroPad2(second)
    third = layer2(third)
    third = BatchNormalization(axis=1, name='l1_bn')(third)
    third = Activation('relu', name='l1_act')(third)

    third = zeroPad3(third)
    third = layer3(third)
    third = BatchNormalization(axis=1, name='l1_bn2')(third)
    third = Activation('relu', name='l1_act2')(third)


    #res = merge([third, second], mode='sum', name='res')
    res = Add(name='res')([third, second])


    first2 = zeroPad1_2(res)
    second2 = layer1_2(first2)
    second2 = BatchNormalization(axis=1, name='major_bn2')(second2)
    second2 = Activation('relu', name='major_act2')(second2)


    third2 = zeroPad2_2(second2)
    third2 = layer2_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn')(third2)
    third2 = Activation('relu', name='l2_act')(third2)

    third2 = zeroPad3_2(third2)
    third2 = layer3_2(third2)
    third2 = BatchNormalization(axis=1, name='l2_bn2')(third2)
    third2 = Activation('relu', name='l2_act2')(third2)

    #res2 = merge([third2, second2], mode='sum', name='res2')
    res2 = Add(name='res2')([third2, second2])
    res2 = Flatten(name='flatten')(res2)

    res2 = layer4(res2)
    res2 = Dropout(0.4, name='dropout1')(res2)
    res2 = layer5(res2)
    res2 = Dropout(0.4, name='dropout2')(res2)
    res2 = final(res2)
    model = Model(inputs=input_img, outputs=res2)

    if binary_classification:
        metrics = ['accuracy', AUC()]
        loss = bc
    else:
        metrics = ['accuracy']
        loss = scc 
    sgd = SGD(decay=0., lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=loss, optimizer=sgd, metrics=metrics)
    return model

In [15]:
X,y=load_nist_by_origin(80000, source='all')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
# tfhub classifier trained with MNIST
tfhub_classifier = hub.KerasLayer("https://tfhub.dev/tensorflow/tfgan/eval/mnist/logits/1", trainable=False)
input_vector = Input(shape=(28,28,1), name="images", dtype=np.float32)
result_vector= tfhub_classifier(input_vector)
tfhub_classifier = Model(inputs=[input_vector], outputs=[result_vector])

In [ ]:
# Own Resnet classifier trained with NIST
res = get_resnet()
history = res.fit(X_train, y_train, validation_split=0.1, verbose=2, epochs=15, batch_size=128)

In [11]:
# keras Resnet50V2 classifier pre-trained with imagenet
resize_width = 32
resize_height = 32
keras_feature_extractor = ResNet50V2(weights='imagenet', include_top=False, input_shape=(resize_width,resize_height,3))

94674944/94668760 [==============================] - 2s 0us/step


In [12]:
X,y = load_nist_by_origin(80000, 'all', resize_width, resize_height)
X_train_keras, X_test_keras, y_train_keras, y_test_keras = train_test_split(X, y, test_size=0.1, random_state=42)
X_test_keras = np.apply_along_axis(grey2RGB,-1,X_test_keras).reshape(X_test_keras.shape[0], resize_width, resize_height,3)
X_train_keras = np.apply_along_axis(grey2RGB,-1,X_train_keras).reshape(X_train_keras.shape[0], resize_width, resize_height,3)

## Digit recognition across different subsets

### MNIST test set

In [21]:
(X_train_mnist, y_train_mnist), (X_test_mnist, y_test_mnist) = mnist.load_data()
assert X_train_mnist.shape == (60000, 28, 28)
assert X_test_mnist.shape == (10000, 28, 28)
assert y_train_mnist.shape == (60000,)
assert y_test_mnist.shape == (10000,)
X_test_mnist = (X_test_mnist/254.).reshape(10000, 28, 28, 1)
X_train_mnist = (X_train_mnist/254.).reshape(60000, 28, 28, 1)

In [18]:
loss_arr=[]
acc_arr=[]
for i in range(10):
    res = get_resnet()
    history = res.fit(X_train, y_train, validation_split=0.1, verbose=0, epochs=15, batch_size=128, shuffle=False)
    loss,acc = res.evaluate(X_test_mnist,y_test_mnist)
    loss_arr.append(loss)
    acc_arr.append(acc)

10000/10000 [==============================] - 1s 130us/sample - loss: 2.1258 - acc: 0.6390


In [19]:
from scipy import stats
stats.describe(acc_arr)

DescribeResult(nobs=10, minmax=(0.5935, 0.7317), mean=0.64772, variance=0.0015455236, skewness=0.8875579237937927, kurtosis=0.24844141244357187)

### NIST High School

In [ ]:
images,labels=load_hsf_nist(4,55000)
res.evaluate(images[10000:],labels[10000:])
#preds = res.predict(images)
#predicted_labels = get_argmax_labels(preds)
#accuracy_score(labels, predicted_labels)

45000/45000 [==============================] - 3s 75us/sample - loss: 0.0525 - acc: 0.9862


[0.05250930038392944, 0.9862222]

In [ ]:
images[10000:].shape

(45000, 28, 28, 1)

### NIST Census MD

In [ ]:
images,labels=load_hsf_nist(7,50000)
res.evaluate(images,labels)

50000/50000 [==============================] - 4s 74us/sample - loss: 0.0376 - acc: 0.9909


[0.03760134117059639, 0.99086]

### NIST Census Field

In [ ]:
images,labels=load_hsf_nist(0,50000)
res.evaluate(images,labels)

50000/50000 [==============================] - 4s 73us/sample - loss: 0.0643 - acc: 0.9851


[0.0642632317849752, 0.98508]

### Synthetic data

In [ ]:
### load generated samples
generate = np.load(os.path.join('/content/drive/MyDrive/Internships/improved_wgan_training/models/wgan-gp_30Ksamples_100kiters', 'generated.npz'))
labels = np.load(os.path.join('/content/drive/MyDrive/Internships/improved_wgan_training/models/wgan-gp_30Ksamples_100kiters', 'generated_labels.npz'))['labels']
gen_imgs = generate['img_r01']
gen_z = generate['noise']
gen_feature = np.reshape(gen_imgs, [len(gen_imgs), -1])
gen_feature = 2. * gen_feature - 1.
gen_feature = gen_feature.reshape(100,28,28,1)

In [ ]:
res.evaluate(gen_feature, labels)

100/100 [==============================] - 0s 672us/sample - loss: 0.4471 - acc: 0.9400


[0.4471098852157593, 0.94]

In [ ]:
predicted_labels = get_argmax_labels(res.predict(gen_feature))

In [ ]:
predicted_labels[90:100]

[8, 5, 6, 5, 2, 7, 6, 3, 7, 9]

In [ ]:
labels.tolist()[90:100]

[8, 5, 6, 5, 2, 7, 6, 3, 7, 9]

## Feature extraction

In [36]:
feature_extractor = Model(inputs=res.input, outputs=res.get_layer('flatten').output)

In [ ]:
feature_clf = RandomForestClassifier()
feature_clf.fit(feature_extractor.predict(X_train), y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_test = feature_clf.predict(feature_extractor.predict(X_test))
accuracy_score(y_test, y_pred_test)

0.9864

In [ ]:
X_train_keras_features = keras_feature_extractor.predict(X_train_keras)
X_train_keras_features = X_train_keras_features.reshape(X_train_keras_features.shape[0], X_train_keras_features.shape[-1])
X_test_keras_features = keras_feature_extractor.predict(X_test_keras)
X_test_keras_features = X_test_keras_features.reshape(X_test_keras_features.shape[0], X_test_keras_features.shape[-1])

In [ ]:
feature_clf = RandomForestClassifier()
feature_clf.fit(X_train_keras_features, y_train_keras)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred = feature_clf.predict(X_test_keras_features)
accuracy_score(y_test, y_pred)

0.8983

In [ ]:
X_test2,y_test2=load_hsf_nist(4,55000)
y_pred_test2 = feature_clf.predict(feature_extractor.predict(X_test2))
accuracy_score(y_test2, y_pred_test2)

0.9915818181818182

## High-school or non high-school recognition

### Keras pre-trained resnet 

In [33]:
resize_width = 32
resize_height = 32
X,y = load_high_school_vs_non(60000, resize_width, resize_height)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_test_keras = np.apply_along_axis(grey2RGB,-1,X_test).reshape(X_test.shape[0], resize_width, resize_height,3)
X_train_keras = np.apply_along_axis(grey2RGB,-1,X_train).reshape(X_train.shape[0], resize_width, resize_height,3)

In [34]:
X_train_keras_features = keras_feature_extractor.predict(X_train_keras)
X_train_keras_features = X_train_keras_features.reshape(X_train_keras_features.shape[0], X_train_keras_features.shape[-1])
X_test_keras_features = keras_feature_extractor.predict(X_test_keras)
X_test_keras_features = X_test_keras_features.reshape(X_test_keras_features.shape[0], X_test_keras_features.shape[-1])

In [38]:
feature_clf = RandomForestClassifier()
feature_clf.fit(X_train_keras_features, y_train_keras)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [39]:
y_pred = feature_clf.predict(X_test_keras_features)
accuracy_score(y_test, y_pred)

0.7176666666666667

### Own resnet

In [46]:
X,y = load_high_school_vs_non(60000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [47]:
res = get_resnet(binary_classification=True)
history = res.fit(X_train, y_train, validation_split=0.1, verbose=2, epochs=15, batch_size=128, shuffle=False)

Train on 48600 samples, validate on 5400 samples
Epoch 1/15
48600/48600 - 7s - loss: 0.6349 - acc: 0.6529 - auc_2: 0.7016 - val_loss: 0.5852 - val_acc: 0.7044 - val_auc_2: 0.7691
Epoch 2/15
48600/48600 - 3s - loss: 0.5866 - acc: 0.6961 - auc_2: 0.7602 - val_loss: 0.5584 - val_acc: 0.7206 - val_auc_2: 0.7965
Epoch 3/15
48600/48600 - 3s - loss: 0.5656 - acc: 0.7144 - auc_2: 0.7825 - val_loss: 0.5516 - val_acc: 0.7230 - val_auc_2: 0.8154
Epoch 4/15
48600/48600 - 3s - loss: 0.5465 - acc: 0.7283 - auc_2: 0.7994 - val_loss: 0.5219 - val_acc: 0.7502 - val_auc_2: 0.8278
Epoch 5/15
48600/48600 - 3s - loss: 0.5286 - acc: 0.7420 - auc_2: 0.8151 - val_loss: 0.5125 - val_acc: 0.7493 - val_auc_2: 0.8379
Epoch 6/15
48600/48600 - 3s - loss: 0.5146 - acc: 0.7515 - auc_2: 0.8268 - val_loss: 0.5128 - val_acc: 0.7541 - val_auc_2: 0.8474
Epoch 7/15
48600/48600 - 3s - loss: 0.5012 - acc: 0.7613 - auc_2: 0.8373 - val_loss: 0.4860 - val_acc: 0.7698 - val_auc_2: 0.8513
Epoch 8/15
48600/48600 - 3s - loss: 0.492

In [48]:
res.evaluate(X_test, y_test)

6000/6000 [==============================] - 1s 141us/sample - loss: 0.5010 - acc: 0.7543 - auc_2: 0.8651


[0.5009915000597636, 0.7543333, 0.8651493]

In [49]:
images1,_ = load_hsf_nist(4,40000)
images1 = images1[40000:]
images2,_ = load_hsf_nist(7,10000)
labels1 = np.array([1 for l in range(images1.shape[0])])
labels2 = np.array([0 for l in range(images2.shape[0])])
res.evaluate(np.concatenate((images1,images2)),np.concatenate((labels1,labels2)))

10000/10000 [==============================] - 1s 142us/sample - loss: 1.3732 - acc: 0.3103 - auc_2: 0.0000e+00


[1.3732483360290528, 0.3103, 0.0]